# Tutorial Retrieval Recency Bias

In this tutorial, we will explore how to test the recency of retrieval results using different LLMs. We will use a sample dataset and a set of test queries to evaluate the performance of various models in retrieving relevant documents based on their recency.

### 1. Import necessary libraries

In [1]:
# Import necessary libraries

import pandas as pd
import os
from dotenv import load_dotenv
from unbiasai.config import DATA_DIR
from unbiasai.utils import initialize_llm, generate_embeddings, insert_documents, retrieve, extract_created_datetime
from supabase import create_client
from queries.retrieval_recency import test_queries
load_dotenv()

# Load data from CSV
file_path = DATA_DIR / 'retrieval_recency.csv'
df = pd.read_csv(file_path)
df = df.head(10)  # For testing, limit to 10 rows


### 2. Initialize models and store them in a dictionary

In [2]:
models = ["gpt", "claude", "mistral", "cohere", "deepseek"]
models = ["gpt", "mistral", "cohere", "deepseek"]

initialized_models = {}

for model_name in models:
    print(model_name)
    initialized_models[model_name] = initialize_llm(model_name)

gpt
Initializing model: gpt
    LLM initialized correctly
mistral
Initializing model: mistral
    LLM initialized correctly
cohere
Initializing model: cohere
    LLM initialized correctly
deepseek
Initializing model: deepseek
    LLM initialized correctly


### 3. Connect to Supabase

In [3]:
# Create Supabase client
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

supabase_client = create_client(SUPABASE_URL, SUPABASE_KEY)

### 4. Generate Embeddings for the Data

Insert your new df including the embeddings in the supabase table to create a vector store.

In [4]:
df['embedding'] = df['content'].apply(generate_embeddings)

# IMPORTANT: change function so supabase table name can be changed.
#
insert_documents(df, supabase_client)


Inserting document with ID: 3984
Inserting document with ID: 3984
Inserting document with ID: 3984
Inserting document with ID: 3984
Inserting document with ID: 3985
Inserting document with ID: 3986
Inserting document with ID: 3986
Inserting document with ID: 3986
Inserting document with ID: 3986
Inserting document with ID: 3987


### 5. Define your Test Queries

In [5]:
#import queries
from queries.retrieval_recency import test_queries

# or define your own queries
test_queries = [
    "What is test query 1?",
    "What is test query 2?",
    "What is test query 3?",
]

### 6. Retrieve and Rerank Documents for Each Query Across All Models

In [6]:
retrieval_results = {}
for model_name, model in initialized_models.items():
    print(f"Running retrieval with model: {model_name}")
    retrieval_results[model_name] = {}
    
    for query in test_queries:
        print(f"  Processing query: {query[:30]}...")
        retrieval_results[model_name][query] = retrieve(
            query, model, supabase_client, function_name='match_documents', k=4, re_rank=True
        )
    
    print(f"✓ Completed all queries for {model_name}")

print("Retrieval complete for all models and queries.")

Running retrieval with model: gpt
  Processing query: What is test query 1?...
Query: What is test query 1?, Embedding: [-0.01123058795928955, 0.02576729655265808, -0.011963387951254845, 0.02670459821820259, -0.021183036267757416, -0.005998735781759024, 0.018183667212724686, 0.048126220703125, -0.014988318085670471, 0.01927434653043747, 0.0130796292796731, 0.0038514609914273024, -0.01896759308874607, -0.023142850026488304, -0.02133641205728054, 0.04594486206769943, -0.02464253269135952, 0.04366125166416168, -0.011366922408342361, -0.02678980864584446, -0.0002620186423882842, 0.02048431895673275, 0.0064631262794137, 0.030044803395867348, 0.021898793056607246, 0.01753607764840126, 0.012764355167746544, 0.002050349023193121, 0.002219702582806349, -0.01601083017885685, -0.018984634429216385, 0.04471784830093384, 0.045774444937705994, -0.025392375886440277, 0.001951293321326375, -0.02719881385564804, -0.009424149990081787, -0.009696819819509983, -0.010182512924075127, 0.021012617275118828, 

APIError: {'code': 'PGRST202', 'details': 'Searched for the function public.match_documents with parameters match_count, query_embedding or with a single unnamed json/jsonb parameter, but no matches were found in the schema cache.', 'hint': 'Perhaps you meant to call the function public.match_documents_recency_no_filter', 'message': 'Could not find the function public.match_documents(match_count, query_embedding) in the schema cache'}

### 6. Process the Rankings
Define 'pattern' to match the 'created' date with regex.

In [7]:
# Initialize a list to collect data
data = []

# Iterate over each model and its corresponding queries
for model, queries in retrieval_results.items():
    for query, documents in queries.items():
        for doc in documents:
            created_datetime = extract_created_datetime(doc['content'], pattern=r'createdDateTime[":]*(\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}(?:\.\d+)?Z)')
            data.append((model, query, doc['rank'], doc['id'], created_datetime))


# Create a DataFrame from the collected data
df = pd.DataFrame(data, columns=['Model', 'Query', 'Rank', 'Document ID', 'Created DateTime'])

Define the date categories

In [8]:
# Define the date categories
date_categories = ['newest', 'newer', 'older', 'oldest']

# Sort and assign date categories within each group
df['date_category'] = (
    df.sort_values(by='Created DateTime', ascending=False)
    .groupby(['Model', 'Query'])
    .cumcount()
    .map({i: category for i, category in enumerate(date_categories)})
)

In [9]:
df.head()

,Model,Query,Rank,Document ID,Created DateTime,date_category
